In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 4.3 MB/s eta 0

In [2]:
!pip install transformers torchvision torch pillow tmm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
from PIL import Image, ImageEnhance
from torchvision import transforms

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = ImageEnhance.Sharpness(image).enhance(2.0)  # Sharpen
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224)
    ])
    return transform(image)

In [4]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import torch.nn.functional as F

processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")
model.eval()

def get_dino_embedding(image_path):
    print("Image path: ", image_path)
    # image = Image.open(image_path).convert("RGB")
    image = preprocess_image(image_path)
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        normalized = torch.nn.functional.normalize(embedding, dim=0)
        return normalized.detach().cpu().numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [5]:
embedding = get_dino_embedding("/content/m1.jpg")
print("Shape: ", embedding.shape)
print("Embedding: ", embedding[:10])

Image path:  /content/m1.jpg
Shape:  (768,)
Embedding:  [-0.05866983  0.02187621 -0.00686383  0.00445763  0.02846012 -0.02582961
  0.01005959 -0.0027339  -0.00990665  0.00271845]


In [6]:
embedding2 = get_dino_embedding("/content/m4.jpg")
print("Shape: ", embedding2.shape)
print("Embedding: ", embedding2[:10])

Image path:  /content/m4.jpg
Shape:  (768,)
Embedding:  [-0.05559961 -0.03526287 -0.03199974 -0.02129219  0.02238583 -0.05490066
  0.00718211 -0.0120649  -0.010157   -0.02341773]


In [7]:
import chromadb
from chromadb.config import Settings
import os

os.environ["CHROMADB_TELEMETRY_ENABLED"] = "false"
client = chromadb.Client()

In [8]:
collection = client.get_or_create_collection(name="cow_muzzle_embeddings")

In [20]:
# collection.delete(where_document={"$contains": "IMG"})

In [9]:
print("Collection count: ", collection.count())

Collection count:  0


In [10]:
image_folder = "/content/muzzles"
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

for idx, filename in enumerate(image_files):
    image_path = os.path.join(image_folder, filename)
    print(f"Processing: {image_path}")
    vector = get_dino_embedding(image_path)
    collection.add(
        ids=[f"img_{idx}"],
        embeddings=[vector],
        metadatas=[{"filename": filename}],
        documents=[image_path]
    )
    print(f"✅ Stored: {filename}")

Processing: /content/muzzles/cow_muzzle_cropped_2.jpg
Image path:  /content/muzzles/cow_muzzle_cropped_2.jpg
✅ Stored: cow_muzzle_cropped_2.jpg
Processing: /content/muzzles/IMG20231118114156.jpg
Image path:  /content/muzzles/IMG20231118114156.jpg
✅ Stored: IMG20231118114156.jpg
Processing: /content/muzzles/m5.jpg
Image path:  /content/muzzles/m5.jpg
✅ Stored: m5.jpg
Processing: /content/muzzles/cow_muzzle_2.jpg
Image path:  /content/muzzles/cow_muzzle_2.jpg
✅ Stored: cow_muzzle_2.jpg
Processing: /content/muzzles/cow_muzzle_cropped_1.jpg
Image path:  /content/muzzles/cow_muzzle_cropped_1.jpg
✅ Stored: cow_muzzle_cropped_1.jpg
Processing: /content/muzzles/m2.jpg
Image path:  /content/muzzles/m2.jpg
✅ Stored: m2.jpg
Processing: /content/muzzles/cow_muzzle.jpg
Image path:  /content/muzzles/cow_muzzle.jpg
✅ Stored: cow_muzzle.jpg
Processing: /content/muzzles/m3.jpg
Image path:  /content/muzzles/m3.jpg
✅ Stored: m3.jpg


In [21]:
query_vector = get_dino_embedding("/content/m2.jpg")
results = collection.query(query_embeddings=[query_vector], n_results=1)

# for doc, meta, distance in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
#     print(f"🔍 Found: {doc} | Metadata: {meta} | Distance: {distance}")

if results["distances"] and results["documents"][0]:
    match_score = 1 - results["distances"][0][0]  # cosine similarity
    percentage = match_score * 100

    print(f"Match Score: {percentage:.2f}%")
    print(f"Match Document: {results['documents'][0][0]}")

    if percentage > 85:
        print("Same cow (Registered)")
    elif percentage > 75:
        print("Possibly same cow, manual review suggested")
    else:
        print("New cow — not in database")
else:
    print("Error: No results found")

Image path:  /content/m2.jpg
Match Score: 37.04%
Match Document: /content/muzzles/m5.jpg
New cow — not in database
